# 작업 1유형

### 1-1
결측치가 하나라도 존재하는 행의 경우 경우 해당 행을 삭제하라. 그후 남은 데이터의 상위 70%에 해당하는 데이터만 남겨둔 후 median_income 컬럼의 1분위수를 반올림하여 소숫점이하 2째자리까지 구하여라

In [1]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e3_p1_1.csv')
df.head(5)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [6]:
df = df.dropna().reset_index(drop=True)
df_filter = df.iloc[:int(len(df)*0.7)]
print(round(df_filter.median_income.quantile(q=0.25),2))

2.51


### 1-2
1990년도는 해당년도 평균 이하 GDP를 가지지만, 2010년도에는 해당년도 평균 이상 GDP를 가지는 국가의 숫자를 구하여라

In [7]:
import pandas as pd
df =pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e3_p1_2.csv')
df.head()

,Country Name,Country Code,Year,Value
0,Arab World,ARB,1968,2576068.3
1,Arab World,ARB,1969,2843420.4
2,Arab World,ARB,1970,3138550.0
3,Arab World,ARB,1971,3642691.0
4,Arab World,ARB,1972,4331605.7


In [28]:
df_1990 = df[df.Year ==1990]
df_2010 = df[df.Year ==2010]


df_1990_filter = df_1990[df_1990.Value <= df_1990.Value.mean()]
df_2010_filter = df_2010[df_2010.Value >= df_2010.Value.mean()]

result = len(set(df_2010_filter['Country Code']) & set(df_1990_filter['Country Code']))
print(result)

5


### 1-3
데이터에서 결측치가 가장 많은 컬럼을 출력하라



In [29]:
import pandas as pd
df = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e3_p1_3.csv')
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3.0,"Braund, Mr. Owen Harris",male,22.0,1.0,0.0,A/5 21171,7.2500,NaN,S
1,2,1,1.0,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1.0,0.0,PC 17599,71.2833,C85,C
2,3,1,3.0,"Heikkinen, Miss. Laina",female,26.0,0.0,0.0,STON/O2. 3101282,7.9250,C85,S
3,4,1,1.0,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1.0,0.0,113803,53.1000,C123,S
4,5,0,3.0,"Allen, Mr. William Henry",male,35.0,0.0,0.0,373450,8.0500,C123,S


In [40]:
result = df.isna().sum().to_frame('na')
print(result.sort_values('na').index[-1])

Fare


# 작업 2유형
TravelInsurance가 1일 확률을 구해서 제출하라. 평가지표 : auc
<br>제출 파일 컬럼 : ID, proba

In [41]:
import pandas as pd
train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e3_p2_train_.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/krdatacertificate/e3_p2_test_.csv')
train.head()

,ID,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad,TravelInsurance
0,1008,26,Private Sector/Self Employed,Yes,1400000,5,0,No,Yes,1
1,199,30,Private Sector/Self Employed,No,1450000,5,0,Yes,Yes,1
2,86,32,Government Sector,Yes,900000,4,0,No,No,0
3,560,26,Private Sector/Self Employed,Yes,1400000,7,0,No,Yes,1
4,161,34,Private Sector/Self Employed,No,1400000,3,1,No,Yes,1


In [45]:
# print(train.isna().sum(), test.isna().sum())  # 결측치 없음
# print(train.info(), test.info())  
# object : Employment Type, GraduateOrNot, FrequentFlyer, EverTravelledAbroad

y = train['TravelInsurance']
train_d = train.drop(columns=['ID','TravelInsurance'])
test_d = test.drop(columns='ID')

train_dd = pd.get_dummies(train_d)
test_dd = pd.get_dummies(test_d)
test_dd = test_dd[train_dd.columns]

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

X_train, X_val, y_train, y_val = train_test_split(train_dd, y, test_size=0.3, stratify=y, random_state=0)
rfc = RandomForestClassifier(random_state=0)
rfc.fit(X_train, y_train)

## 예측, 성능평가
val_proba = rfc.predict_proba(X_val)[:,1]
proba = rfc.predict_proba(test_dd)[:,1]

print('roc_auc_score', roc_auc_score(y_val, val_proba))
pd.DataFrame({'ID':test.ID, 'proba':proba}).to_csv('results/test3_result.csv', index=False)

roc_auc_score 0.8004782494758911


# 작업 3유형